In [1]:
import tensorflow as tf
tf.__version__

'1.15.0'

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
sess = tf.InteractiveSession()

In [4]:
x = tf.placeholder(tf.float32, shape = [None, 784])
y_ = tf.placeholder(tf.float32, shape = [None, 10])

In [5]:
weights = tf.Variable(tf.zeros([784, 10], tf.float32))
bias = tf.Variable(tf.zeros([10], tf.float32))

In [6]:
sess.run(tf.global_variables_initializer())

In [7]:
tf.matmul(x, weights) + bias

<tf.Tensor 'add:0' shape=(?, 10) dtype=float32>

In [8]:
y = tf.nn.softmax(tf.matmul(x, weights) + bias)
y

<tf.Tensor 'Softmax:0' shape=(?, 10) dtype=float32>

In [9]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices = [1]))

In [10]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [11]:
for i in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict = {x: batch[0], y_: batch[1]})

In [12]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
acc = accuracy.eval(feed_dict = {x: mnist.test.images, y_: mnist.test.labels}) * 100
print('final accuracy of simple nn', acc)

final accuracy of simple nn 91.80999994277954


In [13]:
sess.close()

In [14]:
import tensorflow as tf
sess.close()
sess = tf.InteractiveSession()

In [15]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [16]:
dim = 28
flat = dim**2
class_output = 10

In [17]:
x = tf.placeholder(tf.float32, shape = [None, flat])
y_ = tf.placeholder(tf.float32, shape = [None, class_output])

In [18]:
x_image = tf.reshape(x, [-1, 28, 28, 1])
x_image

<tf.Tensor 'Reshape:0' shape=(?, 28, 28, 1) dtype=float32>

In [19]:
w_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev = 0.1))
b_conv1 = tf.Variable(tf.constant(0.1, shape = [32]))

In [20]:
convolve1 = tf.nn.conv2d(x_image, w_conv1, strides = [1, 1, 1, 1], padding = 'SAME')

In [21]:
h_conv1 = tf.nn.relu(convolve1)

In [22]:
conv1 = tf.nn.max_pool(h_conv1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
conv1

<tf.Tensor 'MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>

In [23]:
w_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev = 0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape = [64]))

In [24]:
convolve2 = tf.nn.conv2d(conv1, w_conv2, strides = [1, 1, 1, 1], padding = 'SAME')

In [25]:
h_conv2 = tf.nn.relu(convolve2)

In [26]:
conv2 = tf.nn.max_pool(h_conv2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
conv2

<tf.Tensor 'MaxPool_1:0' shape=(?, 7, 7, 64) dtype=float32>

In [27]:
layer2_matrix = tf.reshape(conv2, [-1, 7 * 7 * 64])

In [28]:
w_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev = 0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape = [1024]))

In [29]:
fc1 = tf.matmul(layer2_matrix, w_fc1) + b_fc1

In [30]:
h_fc1 = tf.nn.relu(fc1)

In [31]:
keep_prob = tf.placeholder(tf.float32)
layer_drop = tf.nn.dropout(h_fc1, keep_prob)
layer_drop

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


<tf.Tensor 'dropout/mul_1:0' shape=(?, 1024) dtype=float32>

In [32]:
w_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev = 0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape = [10]))

In [33]:
fc = tf.matmul(layer_drop, w_fc2) + b_fc2

In [34]:
y_cnn = tf.nn.softmax(fc)
y_cnn

<tf.Tensor 'Softmax_1:0' shape=(?, 10) dtype=float32>

In [35]:
import numpy as np
layer4_test = [[0.9, 0.1, 0.1], [0.9, 0.1, 0.1]]
y_test = [[1.0, 0.0, 0.0], [1.0, 0.0, 0.0]]
np.mean(-np.sum(y_test * np.log(layer4_test), 1))

0.10536051565782628

In [36]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_cnn), reduction_indices = [1]))

In [37]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [38]:
correct_prediction = tf.equal(tf.argmax(y_cnn, 1), tf.argmax(y_, 1))

In [39]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [40]:
sess.run(tf.global_variables_initializer())

In [41]:
for i in range(1100):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict = {x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, float(train_accuracy)))
    train_step.run(feed_dict = {x: batch[0], y_: batch[1], keep_prob: 0.5})

step 0, training accuracy 0.18
step 100, training accuracy 0.86
step 200, training accuracy 0.86
step 300, training accuracy 0.96
step 400, training accuracy 0.94
step 500, training accuracy 0.94
step 600, training accuracy 0.96
step 700, training accuracy 0.94
step 800, training accuracy 0.9
step 900, training accuracy 0.98
step 1000, training accuracy 0.98


In [56]:
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict = {x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict = {x: batch[0], y_: batch[1], keep_prob: 0.5})

step 0, training accuracy 1
step 100, training accuracy 1
step 200, training accuracy 0.98
step 300, training accuracy 1
step 400, training accuracy 0.98
step 500, training accuracy 0.96
step 600, training accuracy 0.94
step 700, training accuracy 1
step 800, training accuracy 0.98
step 900, training accuracy 0.98
step 1000, training accuracy 0.98
step 1100, training accuracy 1
step 1200, training accuracy 1
step 1300, training accuracy 0.96
step 1400, training accuracy 0.96
step 1500, training accuracy 0.98
step 1600, training accuracy 0.96
step 1700, training accuracy 0.98
step 1800, training accuracy 1
step 1900, training accuracy 0.96
step 2000, training accuracy 0.96
step 2100, training accuracy 0.98
step 2200, training accuracy 0.98
step 2300, training accuracy 1
step 2400, training accuracy 1
step 2500, training accuracy 1
step 2600, training accuracy 1
step 2700, training accuracy 1
step 2800, training accuracy 1
step 2900, training accuracy 1
step 3000, training accuracy 1
ste

KeyboardInterrupt: 

In [42]:
n_batches = mnist.test.images.shape[0] // 509
cumulative_acc = 0.0
for i in range(n_batches):
    batch = mnist.test.next_batch(50)
    cumulative_acc += accuracy.eval(feed_dict = {x: batch[0], y_: batch[1], keep_prob: 1.0})
print('test accuracy %d' % (cumulative_acc / n_batches))

test accuracy 0


In [43]:
kernels = sess.run(tf.reshape(tf.transpose(w_conv1, perm = [2, 3, 0, 1]), [32, -1]))

In [45]:
!wget --output-document utils1.py http://deeplearning.net/tutorial/code/utils.py
import utils1
from utils1 import tile_raster_images
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

image = Image.fromarray(tile_raster_images(kernels, img_shape = (5, 5), tile_shape = (4, 8), tile_spacing = (1, 1)))
plt.rcParams['figure.figsize'] = (18.0, 18.0)
imgplot = plt.imshow(image)
imgplot.set_cmap('gray')

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = c:/progra~1/wget/etc/wgetrc
--2020-05-23 10:05:52--  http://deeplearning.net/tutorial/code/utils.py
Resolving deeplearning.net... 132.204.26.28
Connecting to deeplearning.net|132.204.26.28|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5097 (5.0K) [text/plain]
Saving to: `utils1.py'

     0K ....                                                  100% 2.38M=0.002s

2020-05-23 10:05:52 (2.38 MB/s) - `utils1.py' saved [5097/5097]



ModuleNotFoundError: No module named 'PIL'

In [ ]:
import numpy as np

plt.rcParams['figure.figsize'] = (5.0, 5.0)
sampleimage = mnist.test.images[1]
plt.imshow(np.reshape(sampleimage, [28, 28]), cmap = 'gray')

In [ ]:
ActivatedUnits = sess.run(convolve1, feed_dict = {x: np.reshape(sampleimage, [1, 784], order = 'F'), keep_prob: 1.0})
filters = ActivatedUnits.shape[3]
plt.figure(1, figsize = (20, 20))
n_cols = 6
n_rows = np.math.ceil(filters / n_cols) + 1
for i in range(filters):
    plt.sublpot(n_rows, n_cols, i + 1)
    plt.title('Filter ' + str(i))
    plt.imshow(ActivatedUnits[0, :, :, i], interpolation = 'nearest', cmap = 'gray')

In [ ]:
ActivatedUnits = sess.run(convolve2,feed_dict={x:np.reshape(sampleimage,[1,784],order='F'),keep_prob:1.0})
filters = ActivatedUnits.shape[3]
plt.figure(1, figsize=(20,20))
n_columns = 8
n_rows = np.math.ceil(filters / n_columns) + 1
for i in range(filters):
    plt.subplot(n_rows, n_columns, i+1)
    plt.title('Filter ' + str(i))
    plt.imshow(ActivatedUnits[0,:,:,i], interpolation="nearest", cmap="gray")

In [ ]:
sess.close()